In [3]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import sys

In [344]:
image1 = Image.open("paris/paris_b.jpg")
image2 = Image.open("paris/paris_c.jpg")
img1 = np.asarray(image1)
img2 = np.asarray(image2)
#plt.imshow(image)
#points = plt.ginput(5, show_clicks=True)

In [345]:
np.shape(img1)

(480, 640, 3)

In [346]:
with open('paris2-3_15points.npy', 'rb') as f:
    a = np.load(f)
    b = np.load(f)

In [347]:
a

array([[259.33892334, 336.12279287],
       [270.74798341, 337.16041521],
       [288.40523665, 337.52108835],
       [304.68404754, 338.37439401],
       [341.7395648 , 340.7127993 ],
       [383.88752911, 343.91754035],
       [433.71216451, 346.8957472 ],
       [570.60610222, 354.79388259],
       [510.30185556, 366.57470325],
       [352.09389758, 248.26396132],
       [402.62168451, 211.40183442],
       [513.16032765, 207.76719487],
       [556.18713661, 198.7049978 ],
       [598.83602531, 213.08309628],
       [533.12698828, 280.78135563]])

In [348]:
b

array([[ 29.30886914, 320.99214694],
       [ 44.12243353, 322.57542245],
       [ 66.5419748 , 323.53140529],
       [ 86.65703642, 323.74708897],
       [130.43020946, 325.88350403],
       [176.29559   , 326.47232236],
       [226.54647547, 327.95642697],
       [348.2803903 , 331.1072471 ],
       [296.55212848, 345.06854824],
       [145.06883885, 227.18742052],
       [200.60094198, 191.43256777],
       [305.80416225, 196.24711834],
       [343.15420713, 191.72031974],
       [376.9176106 , 206.83630089],
       [319.73490189, 264.69491638]])

In [349]:
def computeH(points_im1, points_im2):
    matrixA = []
    for i in range(len(points_im1)):
        xs, ys = points_im1[i][0], points_im1[i][1]
        xd, yd = points_im2[i][0], points_im1[i][1]
        two_nineMatrix = np.array([[xs, ys, 1,0,0,0, -xd*xs, -xd*ys, -xd],
                                   [0,0,0,xs,ys,1,-yd*xs,-yd*ys,-yd]])
        matrixA.append(two_nineMatrix[0])
        matrixA.append(two_nineMatrix[1])
    return matrixA

In [350]:
A = computeH(b,a)

In [351]:
u, s, vh = np.linalg.svd(A, full_matrices=True)

In [352]:
print(s)
homography = vh[-1].reshape(3,3)
print(homography)
'''
homography = np.array(
    [[ 2.06417031e+00, -1.09152156e-01, -5.91002794e+02],
 [ 4.98137666e-01,  1.73071462e+00, -2.18682112e+02],
 [ 1.70508897e-03, -9.86837491e-05,  1.00000000e+00]])
 '''

[7.59910170e+05 2.16354620e+05 1.35610844e+03 4.93149494e+02
 3.34146896e+02 9.18969921e+01 2.78094370e+01 2.62208784e-01
 4.82691371e-02]
[[-2.33411375e-03  2.66008930e-04 -9.70903563e-01]
 [ 6.76161709e-04 -2.99532242e-03 -2.39405551e-01]
 [ 2.26398995e-06  8.38273143e-07 -4.03805286e-03]]


'\nhomography = np.array(\n    [[ 2.06417031e+00, -1.09152156e-01, -5.91002794e+02],\n [ 4.98137666e-01,  1.73071462e+00, -2.18682112e+02],\n [ 1.70508897e-03, -9.86837491e-05,  1.00000000e+00]])\n '

In [353]:
def interpolation(image, x,y):
    leftTopX, leftTopY, leftTopVal = int(x), int(y), image[int(x)][int(y)]
    rightTopX, rightTopY, rightTopVal = int(x) +1 , int(y), image[int(x)+1][int(y)]
    leftBottomX, leftBottomY, leftBottomVal = int(x), int(y)+1, image[int(x)][int(y)+1]
    rightBottomX, rightBottomY, rightBottomVal = int(x)+1, int(y)+1, image[int(x)+1][int(y)+1]
    a=x%1
    b= int(y)+1 - y
    return leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b

In [354]:
def interpolationNear(image, x,y):
    a=x%1
    b = y%1
    if a >= 0.5:
        if b >= 0.5:
            
    b= int(y)+1 - y
    bilinearInterpolation = leftBottomVal * (1- a)*(1-b) + leftTopVal * (1-a) * b + rightBottomVal * a*(1-b) + rightTopVal * a *b

IndentationError: expected an indented block (5274181.py, line 7)

In [355]:
def pixelIntensity(rgb):
     return 0.299 * rgb[0] + 0.587 * rgb[1] + 0.114 * rgb[2]

In [356]:
def warp(image,homography):
    t_inverse = np.linalg.inv(homography)
    boundries = np.matmul(homography, np.array([[0,0,640,640],
                                                [0,480,0,480],
                                                [1,1,1,1]]))
    boundriesDenormalized = np.matmul(boundries, np.diag(1/boundries[-1]))
    print(boundriesDenormalized, np.shape(boundriesDenormalized))
    inversed = np.matmul(t_inverse, boundriesDenormalized)
    inversedNormalised = np.matmul(inversed, np.diag(1/inversed[-1]))
    print(inversedNormalised)
    newPositions = []
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    minXIndex = np.where(boundriesDenormalized[0] == np.amin(boundriesDenormalized[0]))
    print('minx',minXIndex, boundriesDenormalized[0][minXIndex], boundriesDenormalized[1][minXIndex] )
    coordinates1 = np.matmul(t_inverse, np.array([[-539],[-136],[1]]))
    print("coordinates1",coordinates1 * (1/coordinates1[2][0]) )
    yAxisRange = (int(np.amin(boundriesDenormalized[1])),int(np.amax(boundriesDenormalized[1])))
    xAxisRange = (int(np.amin(boundriesDenormalized[0])),int(np.amax(boundriesDenormalized[0])))
    print(xAxisRange,yAxisRange)
    width = xAxisRange[1]-xAxisRange[0]
    height = yAxisRange[1]-yAxisRange[0]
    wrapped = np.zeros((height+1,width+1,3))
    print(height, width)
    print('traverseIndex',height-1,width-1)
    for h in range(height-1):
        for w in range(width-1):
            x = w + xAxisRange[0]
            y = h + yAxisRange[0]
            coordinates = np.matmul(t_inverse, np.array([[x],[y],[1]]))
            coordinates = (coordinates* (1/coordinates[2][0]))[0:2]
            if coordinates[0] > 0 and coordinates[1] > 0:
                if int(coordinates[1]) < len(image)-1 and int(coordinates[0]) < len(image[0])-1:
                    bilinearInterpolation  = interpolation(image,coordinates[1][0],coordinates[0][0])
                    wrapped[h][w] = bilinearInterpolation
    return np.asarray(wrapped), (xAxisRange[0],yAxisRange[0])

In [357]:
def mergeImages(warped, image, offsets):
    print(offsets)
    xOffset = offsets[0]
    yOffset = offsets[1]
    wWidth = np.shape(warped)[1]
    wHeight = np.shape(warped)[0]
    print(wWidth,wHeight)
    xLimit = wWidth + xOffset
    yLimit = wHeight + yOffset
    mergedX = max(len(image[0]), xLimit)
    mergedY = max(len(image), yLimit)
    shiftImageX = -xOffset if xOffset < 0 else 0
    shiftImageY = -yOffset if yOffset < 0 else 0
    merged = np.zeros(( mergedY + shiftImageY, mergedX + shiftImageX,3))
    print("warpedShape", np.shape(warped), 0 + xOffset, 0 + yOffset)
    print("mergedShape:",np.shape(merged))
    shiftX = 0 if xOffset < 0 else xOffset
    shiftY = 0 if yOffset < 0 else yOffset
    
    for i in range(len(warped)-1):
        for j in range(len(warped[0])-1):
            x = j + shiftX
            y = i + shiftY
            try:
                #a = 8
                merged[y][x] = warped[i][j]
            except:
                print("i,j",i,j, "x,y",x,y)           
            '''
            elif x < len(image[0]) and y < len(image):
                try:
                    merged[i][j] = image[i][j]
                except:
                    print("i,j",i,j, "x,y",x,y)
            else:
                merged[i][j] = warped[i][j]
            
            if x < len(image[0]) and y < len(image) and x > 0 and y > 0:
                try:
                    merged[i][j] = image[x][y]
                except e:
                    print(e)
                    print("i,j",i,j, "x,y",x,y)
            '''
            #elif x < len(image[0]) and y < len(image):           
    print(np.shape(merged),mergedX,mergedY)
    overlapped = warped[-yOffset:,-xOffset:]
    shiftImageX = -xOffset if xOffset < 0 else 0
    shiftImageY = -yOffset if yOffset < 0 else 0
    for i in range(len(image)-1):
        for j in range(len(image[1])-1):
            x = j + shiftImageX
            y = i + shiftImageY
            try:
                imageLuminance = pixelIntensity(image[i][j])
                warpedLuminance = pixelIntensity(merged[y][x])
                #print("i,j",i,j, "x,y",x,y)
                if imageLuminance >= warpedLuminance:
                    merged[y][x] = image[i][j]    
            except:
                print("i,j",i,j, "x,y",x,y) 
    print(np.shape(overlapped))
    #warped[-yOffset:,-xOffset:] = image[:yLimit,:xLimit]
    
    '''
    for i in range(len(overlapped)):
        for j in range(len(overlapped[0])):
    '''        
    return merged

In [358]:
def fwarp(image,homography):
    wrapped = np.zeros(np.shape(image))
    newPositions = []
    for i in range(len(image)):
        for j in range(len(image[i])):
            coordinates = np.matmul(homography, np.array([[i],[j],[1]]))
            newPositions.append(coordinates* (1/coordinates[2][0]))
    return np.asarray(newPositions).reshape(np.shape(np.asarray(image)))[:,:,0:2]

In [359]:
back_res,offset = warp(img2,homography)

[[ 2.40438547e+02  2.31928792e+02  9.51966722e+02  1.06874375e+03]
 [ 5.92873743e+01  4.61305589e+02 -7.46738233e+01  5.69077048e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]] (3, 4)
[[ 5.87614029e-14  5.29061326e-14  6.40000000e+02  6.40000000e+02]
 [-2.93807014e-14  4.80000000e+02  0.00000000e+00  4.80000000e+02]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00  1.00000000e+00]]
minx (array([1]),) [231.92879204] [461.30558916]
coordinates1 [[-3.84947567e+03]
 [-1.58840976e+03]
 [ 1.00000000e+00]]
(231, 1068) (-74, 569)
643 837
traverseIndex 642 836


In [360]:
overlapped = mergeImages(back_res, img1, offset)
print("sonuc", np.shape(overlapped))
imgResult = Image.fromarray(overlapped.astype(np.uint8), 'RGB')
imgResult.save("overlapped.jpg")

(231, -74)
838 644
warpedShape (644, 838, 3) 231 -74
mergedShape: (644, 1069, 3)
(644, 1069, 3) 1069 570
(570, 231, 3)
sonuc (644, 1069, 3)


In [147]:
np.shape(overlapped)

(237, 627, 3)

In [121]:
np.shape(back_res)
 .299r' + .587g' + .114b'

(837, 977, 3)

In [93]:
back_res[1][0]

array([114.74235712, 140.90942738, 167.90942738])

In [120]:
imgResult = Image.fromarray(back_res.astype(np.uint8), 'RGB')
imgResult.save("warpedfirstImage5.jpg")

In [22]:
def construct(image,wrap):
    res = np.round(wrap)
    x_min = np.amin(res[:,:,0])
    y_min = np.amin(res[:,:,1])
    res[:,:,0] = res[:,:,0] + np.abs(x_min)
    res[:,:,1] = res[:,:,1] + np.abs(y_min)
    x_max = np.amax(res[:,:,0]).astype(int)
    y_max = np.amax(res[:,:,1]).astype(int)
    res = res.astype(int)
    print(x_max,y_max)
    scale_X = (len(image) / x_max) 
    scale_Y = (len(image[0]) / y_max)
    projected = np.zeros(np.shape(image))
    for i in range(len(image)):
        for j in range(len(image[i])):
            c = res[i][j]
            #print(int(c[0] * scale_X), int(c[1] * scale_Y) )
            projected[int(c[0] * scale_X)-1][int(c[1] * scale_Y)-1] = image[i][j]
    return projected

In [23]:
imageMatrix=construct(im,res)
imgResult = Image.fromarray(imageMatrix.astype(np.uint8), 'RGB')
imgResult.save("10transform.jpg")

3388 2739


In [24]:
print(np.amin(res[:,:,0]),np.amax(res[:,:,0]))
print(np.amin(res[:,:,1]),np.amax(res[:,:,1]))

-3138.7226735397476 248.77620614530588
-136.12788992140727 2602.759349287685


In [51]:
np.shape(np.array([[1],[2],[3]]))

(3, 1)

In [61]:
int(4.6)

4

In [140]:
3138.72267%1

0.7226700000001074